In [2]:
import pandas as pd 
import numpy as np

In [ ]:
# LC 184 

"""
Employee table:
+----+-------+--------+--------------+
| id | name  | salary | departmentId |
+----+-------+--------+--------------+
| 1  | Joe   | 70000  | 1            |
| 2  | Jim   | 90000  | 1            |
| 3  | Henry | 80000  | 2            |
| 4  | Sam   | 60000  | 2            |
| 5  | Max   | 90000  | 1            |
+----+-------+--------+--------------+
Department table:
+----+-------+
| id | name  |
+----+-------+
| 1  | IT    |
| 2  | Sales |
+----+-------+


T-SQL :
WITH max_sal_cte AS (
    SELECT 
        e.name AS Employee,
        salary AS Salary, 
        d.name AS Department,
        ROW_NUMBER() OVER(PARTITION BY departmentId ORDER BY salary DESC) AS rn
    FROM Employee AS e
    INNER JOIN Department AS d       # left also works
    ON e.departmentId = d.id
)
SELECT Employee, Salary, Department 
FROM max_sal_cte
WHERE rn = 1;
"""


employee_data = {
    'id': [1, 2, 3, 4, 5],
    'name': ['Joe', 'Jim', 'Henry', 'Sam', 'Max'],
    'salary': [70000, 90000, 80000, 60000, 90000],
    'departmentId': [1, 1, 2, 2, 1]
}

department_data = {
    'id': [1, 2],
    'name': ['IT', 'Sales']
}

employee_df = pd.DataFrame(employee_data)
department_df = pd.DataFrame(department_data)


def department_highest_salary(employee: pd.DataFrame, department: pd.DataFrame) -> pd.DataFrame:
    merged_df = employee.merge(department, left_on='departmentId', right_on='id', how='inner', suffixes=['_emp', '_dept'])
    print('merged_df \n', merged_df)
    max_salaries = merged_df.groupby('name_dept')['salary'].transform('max')
    print(max_salaries)
    result_df = merged_df[merged_df['salary'] == max_salaries]
    print(result_df)
    final_df = result_df[['name_dept', 'name_emp', 'salary']]
    final_df.columns = ['Departmen', 'Employee', 'Salary']
    
    return final_df


print(department_highest_salary(employee_df, department_df))

merged_df 
    id_emp name_emp  salary  departmentId  id_dept name_dept
0       1      Joe   70000             1        1        IT
1       2      Jim   90000             1        1        IT
2       3    Henry   80000             2        2     Sales
3       4      Sam   60000             2        2     Sales
4       5      Max   90000             1        1        IT
0    90000
1    90000
2    80000
3    80000
4    90000
Name: salary, dtype: int64
   id_emp name_emp  salary  departmentId  id_dept name_dept
1       2      Jim   90000             1        1        IT
2       3    Henry   80000             2        2     Sales
4       5      Max   90000             1        1        IT
  Departmen Employee  Salary
1        IT      Jim   90000
2     Sales    Henry   80000
4        IT      Max   90000


In [ ]:
# LC 117 - https://leetcode.com/problems/nth-highest-salary/description/

"""
T SQL

CREATE FUNCTION dbo.getNthHighestSalary (@N INT)
RETURNS INT
AS
BEGIN
    -- Return NULL if N is invalid
    IF @N < 1
        RETURN NULL;

    RETURN (
        SELECT DISTINCT(salary)
        FROM Employee
        ORDER BY salary DESC
        OFFSET (@N - 1) ROWS FETCH NEXT 1 ROWS ONLY
    );
END;
"""


data = [[1, 100], [2, 200], [3, 300]]
employee_df = pd.DataFrame(data, columns=['Id', 'Salary']).astype({'Id':'Int64', 'Salary':'Int64'})

def nth_highest_salary(employee: pd.DataFrame, n: int) -> pd.DataFrame:
    sorted_salary = employee['salary'].sort_values(ascending=False)
    unique_values = sorted_salary.unique()
    
    col_name = f'getNthHighestSalary({n})'
    
    if n < 1 or n > len(unique_values):
        return pd.DataFrame({col_name: [None]})
    else:
        nth_val = int(unique_values[n - 1])
        return pd.DataFrame({col_name: [nth_val]})


nth_highest_salary(employee_df, 2)

200


200